In [23]:
import pandas as pd
import numpy as np
from time import sleep
import os.path

In [2]:
artigos = pd.read_csv("titulo_doc_comunidade_page_rank_with_id.csv",delimiter=';',quotechar='\"')
artigos.head()
# artigos.info()

,title,community,id
0,Sidestepping the Triangulation Problem in Baye...,grupo 6,1303.5440v1
1,A Method for Speeding Up Value Iteration in Pa...,grupo 6,1301.6751v1
2,Fast Value Iteration for Goal-Directed Markov ...,grupo 6,1302.1575v1
3,"Incremental Pruning: A Simple, Fast, Exact Met...",grupo 6,1302.1525v1
4,Incremental computation of the value of perfec...,grupo 6,1303.1502v1


In [13]:
artigos.title.duplicated().sum()

0

In [14]:
artigos['downloaded'] = artigos.id.apply(lambda x: os.path.isfile('../pdf/{}.pdf'.format(x)))

In [15]:
artigos[~artigos.downloaded]

,title,community,id,downloaded
194,Selecting Computations: Theory and Applications,grupo 134,1207.5879v1,False


In [27]:
for ID in artigos[~artigos.downloaded].id.unique():
    link = "https://arxiv.org/pdf/"+ID+".pdf"
    ! cd pdf && curl -s -O {link} 
    sleep(np.random.random_sample()*35)

In [10]:
for ID in artigos.id.unique():
    pdf = "../pdf/{}.pdf".format(ID)
    txt = "../txt/{}.txt".format(ID)
    !pdftotext {pdf} {txt}

^C
^C


Syntax Error: Missing language pack for 'Adobe-GB1' mapping
Syntax Error: Unknown font tag 'R126'
Syntax Error (620285): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (620285): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show


^C


Syntax Error: Expected the default config, but wasn't able to find it, or it isn't a Dictionary


##### Remoção das primeiras linhas dos textos para evitar cabeçalho dos artigos

In [5]:
############################## comando para terminal ##############################
# for file in *.txt; do tail -n +20 "$file" -f txt2/"$file"; done

O código a seguir foi adaptado de http://brandonrose.org/clustering#Stopwords,-stemming,-and-tokenizing

In [3]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

In [4]:
############################## Remover comentários dos downloads ao rodar a primeira vez ##############################
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(re.sub('[-\xad][\s\n]','',text)) \
                          for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]{3,}', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
#     return filtered_tokens

[nltk_data] Downloading package stopwords to /home/angelo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/angelo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
tfidf_vectorizer = TfidfVectorizer(input='filename', max_df=0.9, max_features=20000,
                                 min_df=0.05, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

# t = !ls txt2/*.txt
t = !ls ../txt/*.txt

%time tfidf_matrix = tfidf_vectorizer.fit_transform(t)

print(tfidf_matrix.shape)

CPU times: user 2min 27s, sys: 1.88 s, total: 2min 29s
Wall time: 2min 41s
(570, 4275)


In [6]:
terms = tfidf_vectorizer.get_feature_names()
terms[:10]

['aaai',
 'aaai press',
 'aalborg',
 'aalborg denmark',
 'abbrevi',
 'abduct',
 'abil',
 'abl',
 'abov',
 'abov equat']

In [7]:
with open('terms.txt','w') as f:
    f.write('\n'.join(terms))

In [8]:
x = tfidf_matrix.toarray()

In [9]:
df = pd.DataFrame(x,columns=terms,index=[re.sub('../txt?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
df.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,x-axi,y-axi,year,yes,yield,york,zadeh,zero,zero probabl,zhang
1201.2250v1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.016812,0.0,0.008762,0.0,0.000000
1201.2706v2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.009278,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.012576,0.0,0.000000,0.0,0.000000
1201.6655v1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005207,0.0,...,0.0,0.0,0.000000,0.0,0.003463,0.000000,0.0,0.000000,0.0,0.000000
1202.1334v2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.006624,0.004899,0.015579,0.0,...,0.0,0.0,0.000000,0.0,0.015542,0.000000,0.0,0.000000,0.0,0.008658
1202.3698v1,0.021062,0.0,0.0,0.0,0.007406,0.0,0.004745,0.003509,0.000000,0.0,...,0.0,0.0,0.005825,0.0,0.022266,0.000000,0.0,0.000000,0.0,0.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 570 entries, 1201.2250v1 to 1408.2048v1
Columns: 4275 entries, aaai to zhang
dtypes: float64(4275)
memory usage: 18.6+ MB


In [11]:
doc_term_matrix = pd.merge(df,artigos[['community','id']],left_index=True,right_on='id')
doc_term_matrix.reset_index(inplace=True)
doc_term_matrix.drop('level_0',inplace=True,axis=1)
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community,id
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.016812,0.0,0.008762,0.0,0.0,grupo 6,1201.2250v1
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005207,0.0,...,0.000000,0.0,0.003463,0.000000,0.0,0.000000,0.0,0.0,grupo 151,1201.6655v1
2,0.021062,0.000000,0.0,0.0,0.007406,0.0,0.004745,0.003509,0.000000,0.0,...,0.005825,0.0,0.022266,0.000000,0.0,0.000000,0.0,0.0,grupo 134,1202.3698v1
3,0.006916,0.011222,0.0,0.0,0.000000,0.0,0.000000,0.004609,0.000000,0.0,...,0.000000,0.0,0.004874,0.018742,0.0,0.004884,0.0,0.0,grupo 6,1202.3699v1
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.005544,0.028650,0.0,...,0.000000,0.0,0.002931,0.003757,0.0,0.005875,0.0,0.0,grupo 104,1202.3706v1


In [12]:
doc_term_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Columns: 4277 entries, aaai to id
dtypes: float64(4275), object(2)
memory usage: 14.1+ MB


In [13]:
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community,id
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.016812,0.0,0.008762,0.0,0.0,grupo 6,1201.2250v1
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005207,0.0,...,0.000000,0.0,0.003463,0.000000,0.0,0.000000,0.0,0.0,grupo 151,1201.6655v1
2,0.021062,0.000000,0.0,0.0,0.007406,0.0,0.004745,0.003509,0.000000,0.0,...,0.005825,0.0,0.022266,0.000000,0.0,0.000000,0.0,0.0,grupo 134,1202.3698v1
3,0.006916,0.011222,0.0,0.0,0.000000,0.0,0.000000,0.004609,0.000000,0.0,...,0.000000,0.0,0.004874,0.018742,0.0,0.004884,0.0,0.0,grupo 6,1202.3699v1
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.005544,0.028650,0.0,...,0.000000,0.0,0.002931,0.003757,0.0,0.005875,0.0,0.0,grupo 104,1202.3706v1


#### Convertendo os valores dos grupos para números

In [14]:
doc_term_matrix.community.replace(r'[a-zA-Z]* *', value='', inplace=True, regex=True)
doc_term_matrix.community = doc_term_matrix.community.apply(int)
doc_term_matrix.community.unique()

array([  6, 151, 134, 104,  80, 153, 145, 116, 156, 256])

In [15]:
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community,id
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.016812,0.0,0.008762,0.0,0.0,6,1201.2250v1
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005207,0.0,...,0.000000,0.0,0.003463,0.000000,0.0,0.000000,0.0,0.0,151,1201.6655v1
2,0.021062,0.000000,0.0,0.0,0.007406,0.0,0.004745,0.003509,0.000000,0.0,...,0.005825,0.0,0.022266,0.000000,0.0,0.000000,0.0,0.0,134,1202.3698v1
3,0.006916,0.011222,0.0,0.0,0.000000,0.0,0.000000,0.004609,0.000000,0.0,...,0.000000,0.0,0.004874,0.018742,0.0,0.004884,0.0,0.0,6,1202.3699v1
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.005544,0.028650,0.0,...,0.000000,0.0,0.002931,0.003757,0.0,0.005875,0.0,0.0,104,1202.3706v1


In [24]:
doc_term_matrix.community.unique()

array([  6, 151, 134, 104,  80, 153, 145, 116, 156, 256])

#### Discretizando os valores

In [17]:
x2 = doc_term_matrix.drop(['id','community'],axis=1).\
                            replace(to_replace=0,value=np.NaN).\
                            apply(lambda a: pd.qcut(a,3,labels=['baixo','medio','alto']))
x2 = x2.apply(lambda a: a.cat.add_categories(['zero']))
x2['community'] = doc_term_matrix.community
x2.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,y-axi,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,alto,NaN,medio,NaN,NaN,6
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baixo,NaN,...,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN,NaN,151
2,alto,NaN,NaN,NaN,baixo,NaN,baixo,baixo,NaN,NaN,...,NaN,baixo,NaN,alto,NaN,NaN,NaN,NaN,NaN,134
3,medio,medio,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,...,NaN,NaN,NaN,baixo,alto,NaN,baixo,NaN,NaN,6
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medio,alto,NaN,...,NaN,NaN,NaN,baixo,baixo,NaN,medio,NaN,NaN,104


In [18]:
x2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Columns: 4276 entries, aaai to community
dtypes: category(4275), int64(1)
memory usage: 2.5 MB


In [19]:
x2.fillna('zero').to_csv('doc_term_matrix_tfidf_wo_header_discrete.csv',header=True,index=False)

In [20]:
x2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Columns: 4276 entries, aaai to community
dtypes: category(4275), int64(1)
memory usage: 2.5 MB


In [25]:
freq_vectorizer = CountVectorizer(input='filename',max_df=0.9, max_features=20000,
                                 min_df=0.05, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
t = !ls ../txt/*.txt
%time freq_matrix = freq_vectorizer.fit_transform(t)

print(freq_matrix.shape)

CPU times: user 2min 39s, sys: 1.17 s, total: 2min 41s
Wall time: 2min 41s
(570, 4275)


In [26]:
terms_freq = freq_vectorizer.get_feature_names()

with open('terms_tf.txt','w') as f:
    f.write('\n'.join(terms_freq))

In [27]:
x = freq_matrix.toarray()
df = pd.DataFrame(x,columns=terms_freq,index=[re.sub('../txt?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
doc_term_matrix = pd.merge(df,artigos[['community','id']],left_index=True,right_on='id')

doc_term_matrix.set_index('id',inplace=True)
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,y-axi,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community
id,,,,,,,,,,,,,,,,,,,,,
1201.2250v1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,2,0,0,grupo 6
1201.6655v1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,1,0,0,0,0,0,grupo 151
1202.3698v1,4,0,0,0,1,0,1,1,0,0,...,0,1,0,6,0,0,0,0,0,grupo 134
1202.3699v1,1,1,0,0,0,0,0,1,0,0,...,0,0,0,1,3,0,1,0,0,grupo 6
1202.3706v1,0,0,0,0,0,0,0,2,13,0,...,0,0,0,1,1,0,2,0,0,grupo 104


In [38]:
doc_term_matrix.community.replace(r'[a-zA-Z]* *', value='', inplace=True, regex=True)
doc_term_matrix.community = doc_term_matrix.community.apply(int)
doc_term_matrix.community.unique()

array([  6, 151, 134, 104,  80, 153, 145, 116, 156, 256])

In [39]:
doc_term_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 432 entries, 1201.2250v1 to 1407.2510v1
Columns: 4276 entries, aaai to community
dtypes: int64(4276)
memory usage: 14.1+ MB


In [40]:
doc_term_matrix.to_csv('doc_term_matrix_freq_with_stem.csv',header=True)

In [41]:
filtro = doc_term_matrix.drop('community',axis=1)\
                .replace(to_replace=0,value=np.NaN)\
                .apply(lambda a: [-1,0,a.quantile(.75),a.max()])

In [42]:
filtro.head()

aaai                [-1, 0, 2.0, 26.0]
aaai press         [-1, 0, 1.75, 13.0]
aalborg              [-1, 0, 3.0, 7.0]
aalborg denmark      [-1, 0, 2.0, 3.0]
abbrevi              [-1, 0, 1.0, 5.0]
dtype: object

In [43]:
np.sum(filtro.apply(lambda a: len(np.unique(a))) == 4)

4186

In [44]:
x2 = doc_term_matrix.drop(['community'],axis=1)\
            .loc[:,filtro.apply(lambda a: len(np.unique(a))) == 4]\
            .apply(lambda a: pd.cut(a,
                                    [-1,0,a.replace(to_replace=0,value=np.NaN).quantile(.75),a.max()],
                                    labels=['zero','baixo','alto']))
# x2 = x2.apply(lambda a: a.cat.add_categories(['zero']))
x2['community'] = doc_term_matrix.community

x2.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,y-axi,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community
id,,,,,,,,,,,,,,,,,,,,,
1201.2250v1,zero,zero,zero,zero,zero,zero,zero,zero,zero,zero,...,zero,zero,zero,zero,alto,zero,baixo,zero,zero,6
1201.6655v1,zero,zero,zero,zero,zero,zero,zero,zero,baixo,zero,...,zero,zero,zero,baixo,zero,zero,zero,zero,zero,151
1202.3698v1,alto,zero,zero,zero,baixo,zero,baixo,baixo,zero,zero,...,zero,baixo,zero,alto,zero,zero,zero,zero,zero,134
1202.3699v1,baixo,baixo,zero,zero,zero,zero,zero,baixo,zero,zero,...,zero,zero,zero,baixo,alto,zero,baixo,zero,zero,6
1202.3706v1,zero,zero,zero,zero,zero,zero,zero,baixo,alto,zero,...,zero,zero,zero,baixo,baixo,zero,baixo,zero,zero,104


In [45]:
x2.aaai.describe()

count      432
unique       3
top       zero
freq       315
Name: aaai, dtype: object

In [46]:
x2.to_csv('doc_term_matrix_freq_wo_header_discrete.csv',header=True,index=False)

In [47]:
doc_term_matrix.community.value_counts()

153    79
104    66
6      51
134    50
156    44
80     40
256    32
116    32
151    19
145    19
Name: community, dtype: int64

In [48]:
doc_term_matrix.drop('community',axis=1).apply(lambda x: np.corrcoef(x.values,doc_term_matrix.community.values)[0,1])


aaai                            -0.025237
aaai press                      -0.007380
aalborg                          0.387241
aalborg denmark                  0.354521
abbrevi                          0.011277
abduct                          -0.126913
abil                            -0.022043
abl                             -0.021652
abov                            -0.028905
abov equat                       0.007299
abov exampl                     -0.093442
abov result                     -0.064144
absenc                           0.039110
absent                          -0.003392
absolut                         -0.010358
absorb                           0.149576
abstract paper                   0.035297
academ                          -0.020242
academ press                    -0.008663
acceler                         -0.029565
accept                           0.066670
access                          -0.078925
accommod                         0.064729
accomplish                      -0

In [62]:
from scipy.stats import chi2_contingency
chi2_contingency(pd.crosstab(x2.aaai,x2.community).values)

(52.18822524843578,
 3.5004248576802445e-05,
 18,
 array([[48.125     , 23.33333333, 36.45833333, 13.85416667, 13.85416667,
         57.60416667, 32.08333333, 23.33333333, 37.1875    , 29.16666667],
        [13.90277778,  6.74074074, 10.53240741,  4.00231481,  4.00231481,
         16.6412037 ,  9.26851852,  6.74074074, 10.74305556,  8.42592593],
        [ 3.97222222,  1.92592593,  3.00925926,  1.14351852,  1.14351852,
          4.75462963,  2.64814815,  1.92592593,  3.06944444,  2.40740741]]))

In [120]:
b = x2.drop('community',axis=1).apply(lambda x: chi2_contingency(pd.crosstab(x,x2.community))[1])

KeyboardInterrupt: 

In [132]:
x2[b.sort_values(ascending=True)[200:500].index.values.tolist()+['community']].to_csv('doc_term_matrix_freq_chisq_discrete.csv',index=False)

In [133]:
x3 = pd.read_csv('doc_term_matrix_tfidf_discrete.csv')
x3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Columns: 4264 entries, aaai to community
dtypes: int64(1), object(4263)
memory usage: 18.5+ MB


In [134]:
b3 = x3.drop('community',axis=1).apply(lambda x: chi2_contingency(pd.crosstab(x,x3.community))[1])

In [138]:
x3[b3.sort_values(ascending=True)[300:1000].index.values.tolist()+['community']].to_csv('doc_term_matrix_tfidf_chisq_discrete.csv',index=False)